#***Data Processing***

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import random
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
vehicle_data_path = "/content/drive/MyDrive/NativeCarDataset"
os.listdir(vehicle_data_path)

In [ ]:
paths_train=glob.glob(os.path.join(vehicle_data_path,'train/*/*','*.jpg'))
paths_train_xml=glob.glob(os.path.join(vehicle_data_path,'train/*/*','*'))

paths_test=glob.glob(os.path.join(vehicle_data_path,'test/*/*','*.jpg'))
paths_test_xml=glob.glob(os.path.join(vehicle_data_path,'test/*/*','*'))

len(paths_train), len(paths_train_xml), len(paths_test),len(paths_test_xml)

#***Image Classification***

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import cv2
import matplotlib.pyplot as plt
from keras.layers import Input, Lambda, Dense, Flatten

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
vgg = VGG19(input_shape=IMAGE_SIZE +[3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
folders = glob.glob(os.path.join(vehicle_data_path,'train/*'))

In [ ]:
x= Flatten()(vgg.output)
x= Dense(64, activation= 'relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen =ImageDataGenerator(rescale = 1./255,
                                 shear_range = 0.2,
                                 zoom_range = 0.2,
                                 horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/NativeCarDataset/train',
                                                target_size=IMAGE_SIZE,
                                                batch_size= 64,
                                                class_mode='categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/NativeCarDataset/test',
                                           target_size= IMAGE_SIZE,
                                           batch_size = 64,
                                           class_mode = 'categorical')

In [ ]:
len(training_set), len(test_set)

In [ ]:
device_list = tf.test.gpu_device_name()
device_list

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=50,
  validation_steps=len(test_set)
)

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

plt.plot(r.history['accuracy'], label='train accuracy')
plt.plot(r.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf 
from keras.models import load_model
model.save('model_vgg19.h5')